<a href="https://colab.research.google.com/github/hseung67/2023/blob/main/Mission1_1014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⬜ Mission 1 Code

#### **☑ import packages** </font>


In [ ]:
import torch
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
## cuda 사용가능 시 cuda:0 출력

print(device)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import os
import glob
from glob import glob

import cv2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD

In [ ]:
### plotting 한글 깨짐 방지
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf

### 🔷 1-1. (1) 각 메뉴를 클래스로 하는 분류 데이터셋과 데이터로더를 준비

#### ☑ 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### ☑ 데이터 unzip

In [ ]:
### kfood_train file unzip
# !unzip -qq "/content/drive/MyDrive/kfood_train.zip" -d "/content/drive/MyDrive/images"

In [ ]:
#path설정은 각자 설정에 맞게 바꿔주세요
%cd /content/drive/MyDrive/datacamp/train

In [ ]:
ls

#### ☑ labeling(멘토님 알려준 데이터로더 활용 수정)

In [ ]:
#csv 파일 미리 만들기
import os
import pandas as pd

# 이미지 폴더 경로
#image_dir = '/content/drive/MyDrive/datacamp/train'

# CSV 파일 생성
#data = []
#for class_label, class_name in enumerate(os.listdir(image_dir)):
#   class_path = os.path.join(image_dir, class_name)
#    if os.path.isdir(class_path):
#        for image_name in os.listdir(class_path):
#            image_path = os.path.join(class_name, image_name)
#            data.append([image_path, class_label])

#df = pd.DataFrame(data, columns=['file_name', 'label'])
#df.to_csv('/content/drive/MyDrive/datacamp/train/annotations.csv', index=False)


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label

# 이미지 변환을 정의합니다. 여기에서는 이미지 크기를 조정합니다.
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기를 224x224로 조정
    transforms.ToTensor(),         # 이미지를 텐서로 변환
])

# 데이터셋을 생성할 때 이미지 변환을 적용합니다.
training_data = CustomImageDataset(annotations_file='/content/drive/MyDrive/datacamp/train/annotations.csv',
                             img_dir='/content/drive/MyDrive/datacamp/train',
                             transform=image_transform)

In [ ]:
from torchvision import transforms
from PIL import Image
import numpy as np

# 이미지 및 레이블을 불러올 데이터셋을 설정합니다.
training_data = CustomImageDataset(annotations_file='/content/drive/MyDrive/datacamp/train/annotations.csv',
                                  img_dir='/content/drive/MyDrive/datacamp/train',
                                  transform=None)  # 이미지 변환을 사용하지 않습니다.
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]

    # torch.Tensor 이미지를 ndarray로 변환
    img_np = img.permute(1, 2, 0).numpy()

    # ndarray를 PIL 이미지로 변환
    img_pil = Image.fromarray((img_np * 255).astype(np.uint8))

    # 이미지를 시각화합니다.
    figure.add_subplot(rows, cols, i)

    # 이미지 파일 경로를 라벨로 사용
    image_file_path = training_data.img_labels.iloc[sample_idx, 0]
    folder_name = os.path.dirname(image_file_path)

    plt.title(folder_name, y=1.05, fontsize=10, fontweight='bold')
    plt.axis("off")
    plt.imshow(img_pil)

plt.show()




In [ ]:
#데이터로더 적재
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
(여기까지 수정)

(희연 코드)

In [ ]:
# 디렉토리 경로 설정 (이미지 파일이 있는 디렉토리)
# data_directory = "/경로/42개_디렉토리가_있는_상위_디렉토리"
path = "/content/drive/MyDrive/datacamp/train"

# 빈 리스트를 생성하여 데이터(x)와 레이블(y)을 저장할 준비를 합니다.
data = []
labels = []
print('1')
# 42개의 디렉토리에 대한 루프
for directory in os.listdir(path):
    # 각 디렉토리에서 파일 목록을 가져옵니다.
    directory_path = os.path.join(path, directory)
    files = os.listdir(directory_path)

    # 레이블(y)는 디렉토리 이름을 기반으로 생성합니다.
    label = directory  # 레이블은 디렉토리 이름입니다.
    print('2')
    # 각 파일에 대한 루프
    for file in files:
        # 파일 경로 설정
        file_path = os.path.join(directory_path, file)

        # 이미지 파일을 읽어옵니다.
        image = cv2.imread(file_path)
        print('3')
        # 이미지가 존재하는 경우에만 처리합니다.
        if image is not None:
            # 이미지 크기를 조정합니다.
            image = cv2.resize(image, (32, 32))

            # 데이터(x)에 이미지를 추가합니다.
            data.append(image)

            # 레이블(y)에 레이블 값을 추가합니다.
            labels.append(label)

# 데이터와 레이블을 NumPy 배열로 변환합니다.
data = np.array(data)
labels = np.array(labels)

# 데이터(x)와 레이블(y)을 사용하여 분석 및 모델 학습을 위해 활용할 수 있습니다.

In [ ]:
data

In [ ]:
labels

### 🔷 1-1. (2) 예시 이미지를 클래스별로 하나씩 총 42장을 한번에 시각화(plotting)하여 확인


In [ ]:
import matplotlib.pyplot as plt

#### ☑ plotting

In [ ]:
import unicodedata
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

path = "/content/drive/MyDrive/datacamp/train"
foods = sorted(os.listdir(path))
foods = [*map(lambda x: unicodedata.normalize("NFC", x), foods)] # mac 한글 호환 코드

idx = 0

fig, ax = plt.subplots(6, 7, figsize=(20, 15))
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
fig.suptitle("입에서 눈물나는 K-food", y=1.05, fontsize=40, fontweight='bold')

for i in range(6):
    for j in range(7):
        try:
            food = foods[idx]
            idx += 1
        except:
            break  # index 값 벗어나면 끝냄

        while True:
            images = os.listdir(os.path.join(path, food))
            if images:
                random = np.random.choice(images)
                img = plt.imread(os.path.join(path, food, random))
                img = cv2.resize(img, (128, 128))
                ax[i][j].imshow(img)
                ax[i][j].set_title(food, pad=10, fontdict={'fontsize': 15, 'fontweight': 'bold'})
                break
            else:
                # 해당 카테고리에서 이미지를 찾지 못한 경우, 다음 카테고리로 이동
                if idx < len(foods):
                    food = foods[idx]
                    idx += 1
                else:
                    # 모든 카테고리에서 이미지를 찾지 못한 경우 종료
                    break

plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout()
plt.show()

In [ ]:
print(len(images))
print(len(foods))
print(foods)
print(images)


### 🔶 1-2. (1) ResNet18 를 활용하여 42종의 클래스 분류를 수행

#### ☑ ResNet18 Loading

In [ ]:
# from torchvision import models

# import torch.nn as nn
# import torch.nn.functional as F

# resnet_pt = models.resnet18(pretrained=True)
# # freezing
# # for param in resnet_pt.parameters(): ## freezing 제거하여 모델 전체를 재학습
# #     param.requires_grad = False

# # fc layer 수정
# fc_in_features = resnet_pt.fc.in_features
# resnet_pt.fc = nn.Linear(fc_in_features, len(foods))
# resnet_pt = resnet_pt.to(device)

In [ ]:
import torch.nn as nn
import torchvision.models as models

# Load the pretrained ResNet-18 model
resnet_pt = models.resnet18(pretrained=True)

# Modify the first convolution layer to change the number of input channels from 3 to 1
first_conv = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)  # Change 3 input channels to 1

# Concatenate the layers
model = nn.Sequential(first_conv, resnet_pt)

# Replace the last fully connected layer (fc)
fc_in_features = resnet_pt.fc.in_features
model.fc = nn.Linear(fc_in_features, len(foods))

# Move the model to the device (e.g., GPU)
model = model.to(device)

In [ ]:
from torchsummary import summary
summary(model, (3,224,224)) #### 수정함
## total parameter == trainable parameter

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_pt.parameters(), lr=0.001, momentum=0.9)

In [ ]:
resnet_pt.to(device)

#### ☑ Modeling

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
# Assuming you have already obtained 'data' and 'labels' as previously shown

for epoch in range(200):  # Loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(data)):  # Loop over your 'data' and 'labels' arrays
        # Get the inputs and labels from 'data' and 'labels'
        inputs, label = data[i], labels[i]

        # Prepare inputs and labels on the device
        inputs = torch.tensor(inputs).to(device)
        label = torch.tensor(label).to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = resnet_pt(inputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

## Please note:
## 1. You need to make sure that 'data' and 'labels' are in the correct format and that the data and labels correspond correctly.
## 2. This assumes you have imported the necessary libraries, like 'torch' and 'criterion', and have defined 'resnet_pt', 'optimizer', and 'trainloader'.


### 🔶 1-2. (2) Validation 데이터에 대한 정확도를 제시
    ●   ResNet18의 parameters는 무작위로 초기화하여 사용한다.  
        (즉, pretrained weights는 사용할 수 없다.)  
    ●   학습 길이는 50 epoch를 기본으로 하되 추가해도 가능하다.